In [1]:
import sqlite3

In [2]:
import pandas as pd

In [11]:
with sqlite3.connect('C:/Users/poduo/Anaconda3_JupyterNB/20171217Python爬蟲實戰/BOT_currency/BOT_JPY.sqlite') as db:
    df = pd.read_sql_query('select * from BOT_JPY', con = db)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 6 columns):
掛牌日期       129 non-null object
幣別         129 non-null object
現金-本行買入    129 non-null float64
現金-本行賣出    129 non-null float64
即期-本行買入    129 non-null float64
即期-本行賣出    129 non-null float64
dtypes: float64(4), object(2)
memory usage: 6.1+ KB


In [14]:
df.head(5)

,掛牌日期,幣別,現金-本行買入,現金-本行賣出,即期-本行買入,即期-本行賣出
0,2018-01-24 00:00:00,日圓 (JPY),0.2573,0.2683,0.2637,0.2677
1,2018-01-23 00:00:00,日圓 (JPY),0.2553,0.2663,0.2617,0.2657
2,2018-01-22 00:00:00,日圓 (JPY),0.2565,0.2675,0.2629,0.2669
3,2018-01-19 00:00:00,日圓 (JPY),0.2573,0.2683,0.2637,0.2677
4,2018-01-18 00:00:00,日圓 (JPY),0.2573,0.2683,0.2637,0.2677
